In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta


# Yellow cabs

In [134]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')

def fill_with_missing_indicator(df, column_name, fill_value):
    '''
    Function to fill missing rows with certain value and creates informative column
    '''
    new_column = {f'{column_name}_MI': lambda x: np.where(x[column_name]==np.nan, 1, 0)}
    df = df.assign(**new_column)
    df[column_name] = df[column_name].fillna(fill_value) # which value to indicate missingnes
    return df

## missing value imputation with values out of range of standard values, adding binary missing indicator
df = (df
    .pipe(fill_with_missing_indicator, column_name='passenger_count', fill_value=-1)
    .pipe(fill_with_missing_indicator, column_name='RatecodeID', fill_value=0)
    .pipe(fill_with_missing_indicator, column_name='store_and_fwd_flag', fill_value='Missing')
    .pipe(fill_with_missing_indicator, column_name='congestion_surcharge', fill_value=-1)
    .pipe(fill_with_missing_indicator, column_name='airport_fee', fill_value=-1)
    ## also adding simple information about traveling time in minutes
    .assign(trip_time_min=lambda x: (x['tpep_dropoff_datetime']-x['tpep_pickup_datetime'])/timedelta(minutes=1))
)
## removing columns with problematic date, cases where pickup happend after dropoff
df = df.drop(index=df[(df['tpep_dropoff_datetime'] < df['tpep_pickup_datetime'])].index)
## filtering data to find "valid trip cases", explained below
df = (
    df[
        ## trips with 0 traveling, most likely canceled
        ((df['tpep_dropoff_datetime'] >= df['tpep_pickup_datetime']) & (df['trip_distance'] == 0) & (df['PULocationID'] == df['DOLocationID']))
        ## trips between districts, ie they took some time to travel, distance and start end areas changes
        | ((df['tpep_dropoff_datetime'] > df['tpep_pickup_datetime']) & (df['trip_distance'] > 0) & (df['PULocationID'] != df['DOLocationID']))
        ## there may also be trips within a district so lets keep them as long as there is some kind of traveling involved
        | ((df['tpep_dropoff_datetime'] > df['tpep_pickup_datetime']) & (df['trip_distance'] > 0) & (df['PULocationID'] == df['DOLocationID']))
    ]
)

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,passenger_count_MI,RatecodeID_MI,store_and_fwd_flag_MI,congestion_surcharge_MI,airport_fee_MI,trip_time_min
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,1.0,14.30,2.5,0.00,0,0,0,0,0,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,1.0,16.90,2.5,0.00,0,0,0,0,0,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,1.0,34.90,2.5,0.00,0,0,0,0,0,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,1.0,20.85,0.0,1.25,0,0,0,0,0,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,1.0,19.68,2.5,0.00,0,0,0,0,0,10.833333


In [124]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,passenger_count_MI,RatecodeID_MI,store_and_fwd_flag_MI,congestion_surcharge_MI,airport_fee_MI
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,0.0,1.0,14.30,2.5,0.00,0,0,0,0,0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,0.0,1.0,16.90,2.5,0.00,0,0,0,0,0
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,0.0,1.0,34.90,2.5,0.00,0,0,0,0,0
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,0.0,1.0,20.85,0.0,1.25,0,0,0,0,0
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,0.0,1.0,19.68,2.5,0.00,0,0,0,0,0


In [68]:
100029/60/24

69.46458333333334

In [75]:

# (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).min()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,passenger_count_MI,RatecodeID_MI,store_and_fwd_flag_MI,congestion_surcharge_MI,airport_fee_MI
3291,1,2023-01-01 00:11:55,2023-01-01 00:11:55,1.0,0.00,5.0,Y,137,264,2,...,0.0,0.0,0.00,0.0,0.0,0,0,0,0,0
4971,1,2023-01-01 01:54:01,2023-01-01 01:54:01,1.0,0.00,1.0,N,237,264,2,...,0.0,1.0,8.00,2.5,0.0,0,0,0,0,0
11648,1,2023-01-01 02:45:08,2023-01-01 02:45:08,1.0,0.00,1.0,N,234,264,2,...,0.0,1.0,8.00,2.5,0.0,0,0,0,0,0
15664,1,2023-01-01 03:55:38,2023-01-01 03:55:38,1.0,0.00,1.0,N,48,264,2,...,0.0,1.0,8.00,2.5,0.0,0,0,0,0,0
22644,1,2023-01-01 07:24:56,2023-01-01 07:24:56,1.0,0.00,1.0,Y,116,264,2,...,0.0,1.0,14.70,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3057151,2,2023-01-27 17:52:00,2023-01-27 17:52:00,-1.0,3.49,0.0,Missing,239,239,0,...,0.0,1.0,21.48,-1.0,-1.0,0,0,0,0,0
3058374,2,2023-01-28 01:25:00,2023-01-28 01:25:00,-1.0,3.81,0.0,Missing,146,146,0,...,0.0,1.0,20.40,-1.0,-1.0,0,0,0,0,0
3060019,1,2023-01-28 19:20:55,2023-01-28 19:20:55,-1.0,0.00,0.0,Missing,239,239,0,...,0.0,1.0,15.93,-1.0,-1.0,0,0,0,0,0
3062274,2,2023-01-29 18:23:00,2023-01-29 18:23:00,-1.0,0.00,0.0,Missing,107,107,0,...,0.0,1.0,29.28,-1.0,-1.0,0,0,0,0,0


In [22]:
df[df['trip_distance'] == 0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
278,2,2023-01-01 00:39:02,2023-01-01 00:46:03,1.0,0.0,1.0,N,137,162,1,7.90,1.0,0.5,3.22,0.0,1.0,16.12,2.5,0.0
279,2,2023-01-01 00:47:29,2023-01-01 00:55:49,1.0,0.0,1.0,N,233,141,1,8.60,1.0,0.5,2.72,0.0,1.0,16.32,2.5,0.0
280,2,2023-01-01 00:59:24,2023-01-01 01:14:26,1.0,0.0,1.0,N,141,193,2,13.50,1.0,0.5,0.00,0.0,1.0,18.50,2.5,0.0
333,1,2023-01-01 00:57:44,2023-01-01 00:57:59,1.0,0.0,1.0,N,137,137,3,3.00,3.5,0.5,0.00,0.0,1.0,8.00,2.5,0.0
398,2,2023-01-01 00:28:04,2023-01-01 00:28:35,1.0,0.0,2.0,N,142,142,2,70.00,0.0,0.5,0.00,0.0,1.0,74.00,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066753,1,2023-01-31 23:12:06,2023-01-31 23:32:16,NaN,0.0,NaN,None,164,13,0,12.64,0.0,0.5,0.00,0.0,1.0,16.64,NaN,NaN
3066755,1,2023-01-31 23:28:56,2023-01-31 23:45:11,NaN,0.0,NaN,None,144,48,0,13.08,0.0,0.5,0.00,0.0,1.0,17.08,NaN,NaN
3066756,1,2023-01-31 23:05:36,2023-01-31 23:20:37,NaN,0.0,NaN,None,161,148,0,12.74,0.0,0.5,0.00,0.0,1.0,16.74,NaN,NaN
3066758,1,2023-01-31 23:10:56,2023-01-31 23:23:37,NaN,0.0,NaN,None,162,151,0,12.00,1.0,0.5,9.40,0.0,1.0,28.40,NaN,NaN


In [15]:
df = pd.read_parquet('../data/green_tripdata_2023-01.parquet')

In [16]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.90,1.0,0.5,4.03,0.0,None,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.70,1.0,0.5,2.64,0.0,None,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.20,1.0,0.5,1.94,0.0,None,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.50,0.5,1.5,1.70,0.0,None,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.00,0.5,1.5,0.00,0.0,None,1.0,8.00,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68206,2,2023-01-31 22:29:00,2023-01-31 22:42:00,None,NaN,49,62,NaN,4070.82,15.70,0.0,0.0,0.00,0.0,None,1.0,16.70,NaN,NaN,NaN
68207,2,2023-01-31 22:40:00,2023-01-31 22:48:00,None,NaN,10,205,NaN,2.14,4.41,0.0,0.0,0.00,0.0,None,1.0,5.41,NaN,NaN,NaN
68208,2,2023-01-31 23:46:00,2023-02-01 00:02:00,None,NaN,66,37,NaN,3.44,16.53,0.0,0.0,3.51,0.0,None,1.0,21.04,NaN,NaN,NaN
68209,2,2023-01-31 23:01:00,2023-01-31 23:19:00,None,NaN,225,189,NaN,3.03,14.98,0.0,0.0,3.20,0.0,None,1.0,19.18,NaN,NaN,NaN
